In [2]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from pandas.io.json import json_normalize   
import pandas as pd  
import time
import json
import re

In [3]:
my_url = "http://www.trentotoday.it/search/from/01.01.2010/to/27.02.2020/model/articolo"

# Opening up connection, grabbing the page
UClient =  uReq(my_url)
time.sleep(3)
page_html = UClient.read()

In [4]:
 # html parser
page_soup = soup(page_html,"lxml")

In [4]:
#stories = page_soup.find("ul",{"class","u-unstyled search-results"})

In [17]:
#articles = stories.findAll("li",{"class","result-item nw_result_articolo"})

In [5]:
# Cerco il numero di pagine per ciclare su ognuna di esse
temp = page_soup.find("div",{"class", "form-group search-input"})
n_pages = temp.find_all("strong")[1].get_text()

In [45]:
# Creo il database
columns = ['title', 'link', 'timestamp', 'text', 'keywords' ,'address', 'gps']
df = pd.DataFrame(columns=columns)


In [46]:
url_search = 'http://www.trentotoday.it/search/model/articolo/from/01.01.2010/to/27.02.2020/pag/'

# Per ogni pagina mi prendo le informazioni
for i in range(1,int(n_pages)):
    url_complete = url_search + str(i) #creo il link
    UClient =  uReq(url_complete)
    time.sleep(2)
    html = UClient.read()

    page_soup = soup(html,"lxml")
    stories = page_soup.find("ul",{"class","u-unstyled search-results"})
    articles = stories.findAll("li",{"class","result-item nw_result_articolo"})

    for art in articles:
        if(art.article.div.header.p.span.get_text()=="Articolo"):
            title = art.a.get_text()
            link = 'http://www.trentotoday.it{}'.format(art.a['href'])
            timestamp = art.i.get_text()
            #print(title[20:-18], link, timestamp)
            df = df.append({'title':title[20:-18], 'link':link, 'timestamp':timestamp ,'text':'', 'keywords':'', 'address':'', 'gps':''},ignore_index = True)

                

In [47]:
print(df)

title  \
0     Minacce al leader di Casa Pound Trento, invo...   
1     A Lavis arrivano 11 social house: ecco come ...   
2     Chico Forti, si riaccende la speranza: "Di M...   
3     Il Trentino della solidarietà in lutto: addi...   
4       Travolto da un albero: muore boscaiolo 17enne   
5     Lavori di restauro per i "Tre portoni": ecco...   
6     Polizia stradale: in 10 giorni via 37 patent...   
7     Ragazzo di 20 anni trovato morto in un palaz...   
8     L'Alto Adige riapre le scuole, in Trentino s...   
9     Coronavirus: due studenti trentini isolati i...   
10    Coronavirus: i musei provinciali non sono ch...   
11    Vastissimo incendio, la montagna brucia da d...   
12    Associazione donatori midollo osseo, il Tren...   
13    Auto a zig-zag in paese, arrivano i vigili: ...   
14    Amuchina "fai da te"? Non fatelo a casa: ecc...   
15    Coronavirus: sanificazione dei mezzi pubblic...   
16    Pauroso incidente a Mori, schianto frontale ...   
17    Università di Tr

In [48]:
df.to_csv("prova.csv",header = True)